# **Deforum Stable Diffusion**
[Stable Diffusion](https://github.com/CompVis/stable-diffusion) by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer and the [Stability.ai](https://stability.ai/) Team

Notebook by [deforum](https://twitter.com/deforum_art)

Changes by [Thibaudz](https://twitter.com/thibaudz): 

- the seed is incremented by 1 between each iteration and save in the name of each file
- grid without space between image
- autoset ddim_eta to 0 when plms is checked
- f (downsampling factor) default to 8 (so the size of the image are really the one filled and not x2)
- skip env setup checkbox
- multi prompt
- more samplers ("ddim","plms","k_lms","dpm_2","dpm_2_ancestral","heun","euler","euler_ancestral")


# 1 Setup

In [ ]:
#@markdown **GPU**
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
#@markdown **Env Setup**

skip_env_setup = True #@param {type:"boolean"}

if not skip_env_setup:
  #@markdown Runtime > Restart Runtime
  %pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
  %pip install omegaconf==2.1.1 einops==0.3.0 pytorch-lightning==1.4.2 torchmetrics==0.6.0 torchtext==0.2.3 transformers==4.19.2 kornia==0.6
  !git clone https://github.com/deforum/stable-diffusion
  #%pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  %pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
  %pip install git+https://github.com/crowsonkb/k-diffusion/
  print("ici")
else:
  print("Setup env [SKIPPED]")

In [ ]:
#@markdown **Local Path Variables**
models_path = "models/ldm/stable1p4b" #@param {type:"string"}
output_path = "/mnt/c/Users/Thiba/Downloads/output_colab" #@param {type:"string"}

#@markdown **Google Drive Path Variables (Optional)**
mount_google_drive = False #@param {type:"boolean"}
force_remount = False

if mount_google_drive:
  from google.colab import drive
  try:
    drive_path = "/content/drive"
    drive.mount(drive_path,force_remount=force_remount)
    models_path = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
    output_path = "/content/drive/MyDrive/AI/StableDiffusion" #@param {type:"string"}
  except:
    print("...error mounting drive or with drive path variables")
    print("...reverting to default path variables")
    models_path = "/content/models"
    output_path = "/content/output"

!mkdir -p $models_path
!mkdir -p $output_path

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")

In [ ]:
#@markdown **Python Definitions**
import json
from IPython import display
#from tqdm.notebook import tqdm, trange

#from google.colab import output
#output.enable_custom_widget_manager()

import sys, os
import argparse, glob
import torch
import torch.nn as nn
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange, repeat
from torchvision.utils import make_grid
import time
from pytorch_lightning import seed_everything
from torch import autocast
from contextlib import contextmanager, nullcontext

sys.path.append('./stable-diffusion/')

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

import k_diffusion as K
import accelerate

#import for animation
import pandas as pd
import subprocess
from glob import glob
from resize_right import resize
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2


def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

def get_output_folder(output_path,batch_folder=None):
  yearMonth = time.strftime('%Y-%m/')
  out_path = output_path+"/"+yearMonth
  if batch_folder != "":
    out_path += batch_folder
    if out_path[-1] != "/":
      out_path += "/"
  os.makedirs(out_path, exist_ok=True)
  return out_path

def load_img(path,w,h):
    image = Image.open(path).convert("RGB")
    #w, h = image.size
    image = TF.resize(image, min(w, h, *image.size), T.InterpolationMode.LANCZOS)
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
    image = image.resize((w, h), resample=Image.LANCZOS)
    image = np.array(image).astype(np.float16) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

class CFGDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, uncond, cond, cond_scale):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])
        uncond, cond = self.inner_model(x_in, sigma_in, cond=cond_in).chunk(2)
        return uncond + (cond - uncond) * cond_scale

def run(params):

    # outpath
    os.makedirs(params["outdir"], exist_ok=True)
    outpath = params["outdir"]

    # timestring
    timestring = time.strftime('%Y%m%d%H%M%S')

    # random seed
    if params["seed"] == -1:
      local_seed = np.random.randint(0,4294967295)
    else:
      local_seed = params["seed"]

    # save/append settings
    if params["save_settings"] and params["filename"] is None:
      filename = f"{timestring}_settings.txt"
      assert not os.path.isfile(f"{outpath}{filename}")
      params["filename"] = f"{timestring}_settings.txt"
      params["batch_seeds"] = [local_seed]
      with open(f"{outpath}{filename}", "w+") as f:
        json.dump(params, f, ensure_ascii=False, indent=4)
    elif params["save_settings"] and params["filename"] is not None:
      filename = params["filename"]
      with open(f"{outpath}{filename}") as f:
        params = json.load(f)
      params["batch_seeds"] += [local_seed]
      with open(f"{outpath}{filename}", "w+") as f:
        json.dump(params, f, ensure_ascii=False, indent=4)

    # load settings

    accelerator = accelerate.Accelerator()
    device = accelerator.device
    seeds = torch.randint(-2 ** 63, 2 ** 63 - 1, [accelerator.num_processes])
    torch.manual_seed(seeds[accelerator.process_index].item())

    # seed
    seed_everything(local_seed)


    # plms
    if params["sampler"]=="plms":
        params["ddim_eta"] = 0
        sampler = PLMSSampler(model)
    else:
        sampler = DDIMSampler(model)

    model_wrap = K.external.CompVisDenoiser(model)
    sigma_min, sigma_max = model_wrap.sigmas[0].item(), model_wrap.sigmas[-1].item()

    batch_size = params["n_samples"]
    n_rows = params["n_rows"] if params["n_rows"] > 0 else batch_size

    if params["key_frames"]:
      angle_series = get_inbetweens(parse_key_frames(params["angle"]))
      zoom_series = get_inbetweens(parse_key_frames(params["zoom"]))
      translation_x_series = get_inbetweens(parse_key_frames(params["translation_x"]))
      translation_y_series = get_inbetweens(parse_key_frames(params["translation_y"]))

    #if not params["from_file"]:
    #    prompt = params["prompt"]
    #    assert prompt is not None
    #    data = [batch_size * [prompt]]
    #else:
    #    print(f"reading prompts from {from_file}")
    #    with open(from_file, "r") as f:
    #        data = f.read().splitlines()
    #        data = list(chunk(data, batch_size))

    print(params["prompts"])

    data = list(chunk(params["prompts"], batch_size))


    sample_path = os.path.join(outpath, "samples")
    os.makedirs(sample_path, exist_ok=True)
    #base_count = len(os.listdir(sample_path))
    base_count = 0

    start_code = None
    print(params)
    if params["animation_mode"] == "None":
      params["max_frames"] = 1
    else:
      A=1 #params["max_frames"] = 24

    for frame_num in range(params["max_frames"]):   

      #Animation
      if  params["animation_mode"] == "Video Input":
        display.clear_output(wait=True)
        #if not video_init_seed_continuity:
        #  seed += 1
        params["use_init"] = True
        params["init_image"] = f'{params["videoFramesFolder"]}/{frame_num+1:04}.jpg'
        params['strength'] = params["strength_previous_frame"]   


      elif params["animation_mode"] == "2D":
        display.clear_output(wait=True)
        if frame_num > 0:
          params["use_init"] = True
          params["init_image"] = os.path.join(outpath, f"samples/{timestring}_{(base_count-1):04}_seed_{local_seed2}.png")
          params['strength'] = params["strength_previous_frame"]
          local_seed2 += 1000

          img_0 = cv2.imread(params["init_image"])

          if params["key_frames"]:
            angle = angle_series[frame_num]
            zoom = zoom_series[frame_num]
            translation_x = translation_x_series[frame_num]
            translation_y = translation_y_series[frame_num]
            print(
                f'angle: {angle}',
                f'zoom: {zoom}',
                f'translation_x: {translation_x}',
                f'translation_y: {translation_y}',
            )
          
          center = (1*img_0.shape[1]//2, 1*img_0.shape[0]//2)
          trans_mat = np.float32(
              [[1, 0, translation_x],
              [0, 1, translation_y]]
          )
          rot_mat = cv2.getRotationMatrix2D( center, angle, zoom )
          trans_mat = np.vstack([trans_mat, [0,0,1]])
          rot_mat = np.vstack([rot_mat, [0,0,1]])
          transformation_matrix = np.matmul(rot_mat, trans_mat)
          img_0 = cv2.warpPerspective(
              img_0,
              transformation_matrix,
              (img_0.shape[1], img_0.shape[0]),
              borderMode=cv2.BORDER_WRAP
          )
          cv2.imwrite('prevFrameScaled.png', img_0)
          params["init_image"] = 'prevFrameScaled.png'
          

      # init image
      if params["use_init"]:
        assert os.path.isfile(params["init_image"])

        init_image = load_img(params["init_image"], params["W"], params["H"]).to(device)
        init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
        init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space

        sampler.make_schedule(ddim_num_steps=params['ddim_steps'], ddim_eta=params['ddim_eta'], verbose=False)
    
        assert 0. <= params['strength'] <= 1., 'can only work with strength in [0.0, 1.0]'
        t_enc = int(params['strength'] * params['ddim_steps'])
        print(f"target t_enc is {t_enc} steps")

      # no init image
      else:
        if params["fixed_code"]:
            start_code = torch.randn([params["n_samples"], params["C"], params["H"] // params["f"], params["W"] // params["f"]], device=device)

      precision_scope = autocast if params["precision"]=="autocast" else nullcontext
      with torch.no_grad():
          with precision_scope("cuda"):
              with model.ema_scope():
                  tic = time.time()
                  all_samples = list()
                  #for n in trange(params["n_iter"], desc="Sampling"):
                  for prompts in data:
                      all_samples = list()
                      grid_count = len(os.listdir(outpath)) - 1
                      for n in range(params["n_iter"]):
                          local_seed2 = local_seed + n
                          seed_everything(local_seed2)
                          
                          if params["sampler"]=="plms":
                            sampler = PLMSSampler(model)
                            if params["use_init"]:
                              sampler.make_schedule(ddim_num_steps=params['ddim_steps'], ddim_eta=params['ddim_eta'], verbose=False)
                          #for prompts in tqdm(data, desc="data"):
                          uc = None
                          if params["scale"] != 1.0:
                              uc = model.get_learned_conditioning(batch_size * [""])
                          if isinstance(prompts, tuple):
                              prompts = list(prompts)
                          c = model.get_learned_conditioning(prompts)
    #sampler = "ddim" # @param ["ddim","plms","k_lms","dpm_2","dpm_2_ancestral","heun","euler","euler_ancestral"] 

                          if params["sampler"] in ["k_lms","dpm_2","dpm_2_ancestral","heun","euler","euler_ancestral"]:
                            shape = [params["C"], params["H"] // params["f"], params["W"] // params["f"]]
                            sigmas = model_wrap.get_sigmas(params["ddim_steps"])
                            torch.manual_seed(local_seed2)
                            x = torch.randn([params["n_samples"], *shape], device=device) * sigmas[0]
                            model_wrap_cfg = CFGDenoiser(model_wrap)
                            extra_args = {'cond': c, 'uncond': uc, 'cond_scale': params["scale"]}
                            if params["sampler"]=="k_lms":
                              samples_ddim = K.sampling.sample_lms(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=not accelerator.is_main_process)
                            elif params["sampler"]=="dpm_2":
                              samples_ddim = K.sampling.sample_dpm_2(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=not accelerator.is_main_process)
                            elif params["sampler"]=="dpm_2_ancestral":
                              samples_ddim = K.sampling.sample_dpm_2_ancestral(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=not accelerator.is_main_process)
                            elif params["sampler"]=="heun":
                              samples_ddim = K.sampling.sample_heun(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=not accelerator.is_main_process)
                            elif params["sampler"]=="euler":
                              samples_ddim = K.sampling.sample_euler(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=not accelerator.is_main_process)
                            elif params["sampler"]=="euler_ancestral":
                              samples_ddim = K.sampling.sample_euler_ancestral(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=not accelerator.is_main_process)
                              
                            x_samples_ddim = model.decode_first_stage(samples_ddim)
                            x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
                            x_samples_ddim = accelerator.gather(x_samples_ddim)
                            x_samples = x_samples_ddim

                          else:

                            # no init image
                            if not params['use_init']:
                                shape = [params["C"], params["H"] // params["f"], params["W"] // params["f"]]

                                samples_ddim, _ = sampler.sample(S=params["ddim_steps"],
                                                                conditioning=c,
                                                                batch_size=params["n_samples"],
                                                                shape=shape,
                                                                verbose=False,
                                                                unconditional_guidance_scale=params["scale"],
                                                                unconditional_conditioning=uc,
                                                                eta=params["ddim_eta"],
                                                                x_T=start_code)

                                x_samples_ddim = model.decode_first_stage(samples_ddim)
                                x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
                                x_samples = x_samples_ddim

                            # init image
                            else:
                              # encode (scaled latent)
                              z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))
                              # decode it
                              samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=params['scale'],
                                                      unconditional_conditioning=uc,)
      
                              x_samples = model.decode_first_stage(samples)
                              x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
                          
                          for x_sample in x_samples:
                              x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                              if params["display_samples"]:
                                  display.display(Image.fromarray(x_sample.astype(np.uint8)))
                              if not params["skip_save"]:
                                  if params["animation_mode"] != "None":
                                    Image.fromarray(x_sample.astype(np.uint8)).save(
                                        os.path.join(outpath, f"samples/{timestring}_{base_count:04}_seed_{local_seed}.png"))
                                  else:
                                    Image.fromarray(x_sample.astype(np.uint8)).save(
                                        os.path.join(outpath, f"samples/{timestring}_{base_count:04}_seed_{local_seed2}.png"))
                              base_count += 1

                          if not params["skip_grid"]:
                              all_samples.append(x_samples)

                      if not params["skip_grid"]:
                          # additionally, save as grid
                          grid = torch.stack(all_samples, 0)
                          grid = rearrange(grid, 'n b c h w -> (n b) c h w')
                          grid = make_grid(grid, nrow=n_rows,padding=0)

                          # to image
                          grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
                          if params["display_grid"]:
                              display.display(Image.fromarray(grid.astype(np.uint8)))
                          Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'{timestring}_{grid_count:04}_grid.png'))
                          #grid_count += 1

                  toc = time.time()


    #Animation - Create Video
    # import subprocess in case this cell is run without the above cells
    if  params["animation_mode"] != "None":
      import subprocess
      from base64 import b64encode
      init_frame = 1
      fps = 12
      last_frame = params["max_frames"]
      image_path = f"{outpath}/samples/{timestring}_%04d_seed_{local_seed}.png"
      file_path = os.path.join(outpath, f'{timestring}.mp4')
      cmd = [
        'ffmpeg',
        '-y',
        '-vcodec',
        'png',
        '-r',
        str(fps),
        '-start_number',
        str(init_frame),
        '-i',
        image_path,
        '-frames:v',
        str(last_frame+1),
        '-c:v',
        'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt',
        'yuv420p',
        '-crf',
        '17',
        '-preset',
        'veryslow',
        file_path
      ]

      process = subprocess.Popen(cmd, cwd=f'{outpath}/samples', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      stdout, stderr = process.communicate()
      if process.returncode != 0:
          print(stderr)
          raise RuntimeError(stderr)
      else:
          print("The video is ready and saved to the images folder")


    #print(f"Your samples are ready and waiting for you here: \n{outpath} \n" f" \nEnjoy.")


def parse_key_frames(string, prompt_parser=None):
    """Given a string representing frame numbers paired with parameter values at that frame,
    return a dictionary with the frame numbers as keys and the parameter values as the values.

    Parameters
    ----------
    string: string
        Frame numbers paired with parameter values at that frame number, in the format
        'framenumber1: (parametervalues1), framenumber2: (parametervalues2), ...'
    prompt_parser: function or None, optional
        If provided, prompt_parser will be applied to each string of parameter values.
    
    Returns
    -------
    dict
        Frame numbers as keys, parameter values at that frame number as values

    Raises
    ------
    RuntimeError
        If the input string does not match the expected format.
    
    Examples
    --------
    >>> parse_key_frames("10:(Apple: 1| Orange: 0), 20: (Apple: 0| Orange: 1| Peach: 1)")
    {10: 'Apple: 1| Orange: 0', 20: 'Apple: 0| Orange: 1| Peach: 1'}

    >>> parse_key_frames("10:(Apple: 1| Orange: 0), 20: (Apple: 0| Orange: 1| Peach: 1)", prompt_parser=lambda x: x.lower()))
    {10: 'apple: 1| orange: 0', 20: 'apple: 0| orange: 1| peach: 1'}
    """
    import re
    pattern = r'((?P<frame>[0-9]+):[\s]*[\(](?P<param>[\S\s]*?)[\)])'
    frames = dict()
    for match_object in re.finditer(pattern, string):
        frame = int(match_object.groupdict()['frame'])
        param = match_object.groupdict()['param']
        if prompt_parser:
            frames[frame] = prompt_parser(param)
        else:
            frames[frame] = param

    if frames == {} and len(string) != 0:
        raise RuntimeError('Key Frame string not correctly formatted')
    return frames

def get_inbetweens(key_frames, integer=False):

    """Given a dict with frame numbers as keys and a parameter value as values,
    return a pandas Series containing the value of the parameter at every frame from 0 to max_frames.
    Any values not provided in the input dict are calculated by linear interpolation between
    the values of the previous and next provided frames. If there is no previous provided frame, then
    the value is equal to the value of the next provided frame, or if there is no next provided frame,
    then the value is equal to the value of the previous provided frame. If no frames are provided,
    all frame values are NaN.

    Parameters
    ----------
    key_frames: dict
        A dict with integer frame numbers as keys and numerical values of a particular parameter as values.
    integer: Bool, optional
        If True, the values of the output series are converted to integers.
        Otherwise, the values are floats.
    
    Returns
    -------
    pd.Series
        A Series with length max_frames representing the parameter values for each frame.
    
    Examples
    --------
    >>> max_frames = 5
    >>> get_inbetweens({1: 5, 3: 6})
    0    5.0
    1    5.0
    2    5.5
    3    6.0
    4    6.0
    dtype: float64

    >>> get_inbetweens({1: 5, 3: 6}, integer=True)
    0    5
    1    5
    2    5
    3    6
    4    6
    dtype: int64
    """
    key_frame_series = pd.Series([np.nan for a in range(params["max_frames"])])

    for i, value in key_frames.items():
        key_frame_series[i] = value
    key_frame_series = key_frame_series.astype(float)
    
    print(params["interp_spline"])
    interp_method = params["interp_spline"]

    if interp_method == 'Cubic' and len(key_frames.items()) <=3:
      interp_method = 'Quadratic'
    
    if interp_method == 'Quadratic' and len(key_frames.items()) <= 2:
      interp_method = 'Linear'
      
    
    key_frame_series[0] = key_frame_series[key_frame_series.first_valid_index()]
    key_frame_series[params["max_frames"]-1] = key_frame_series[key_frame_series.last_valid_index()]
    # key_frame_series = key_frame_series.interpolate(method=intrp_method,order=1, limit_direction='both')
    key_frame_series = key_frame_series.interpolate(method=interp_method.lower(),limit_direction='both')
    if integer:
        return key_frame_series.astype(int)
    return key_frame_series


In [ ]:
#@markdown **Model Path Variables**
# ask for the link
download_link = "" #@param {type:"string"}

# config
if os.path.exists(models_path+'/v1-inference.yaml'):
  print(f"{models_path+'/v1-inference.yaml'} exists")
else:
  print("cp ./stable-diffusion/configs/stable-diffusion/v1-inference.yaml $models_path/.")
  !cp ./stable-diffusion/configs/stable-diffusion/v1-inference.yaml $models_path/


# weights 
if os.path.exists(models_path+'/sd-v1-3-full-ema.ckpt'):
  print(f"{models_path+'/sd-v1-3-full-ema.ckpt'} exists")
else:
  print(f"!wget -O $models_path/sd-v1-3-full-ema.ckpt {download_link}")
  !wget -O $models_path/sd-v1-3-full-ema.ckpt $download_link

config = models_path+'/v1-inference.yaml'
ckpt = models_path+'/sd-v1-3-full-ema.ckpt'

print(f"config: {config}")
print(f"ckpt: {ckpt}")

In [ ]:
#@markdown **Load Stable Diffusion**

def load_model_from_config(config, ckpt, verbose=False, device='cuda'):
    map_location = "cuda" #@param ["cpu", "cuda"]
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location=map_location)
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    #model.cuda()
    model = model.half().to(device)
    model.eval()
    return model

load_on_run_all = False #@param {type: 'boolean'}

if load_on_run_all:
  local_config = OmegaConf.load(f"{config}")
  model = load_model_from_config(local_config, f"{ckpt}")
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model = model.to(device)

# **Run**

In [ ]:
def opt_params():
  
  #@markdown **Save & Display Settings**
  batchdir = "test" #@param {type:"string"}
  outdir = get_output_folder(output_path,batchdir)
  save_settings = True #@param {type:"boolean"}
  skip_save = False #@param {type:"boolean"}
  skip_grid = False #@param {type:"boolean"}
  display_grid = True #@param {type:"boolean"}
  display_samples = True #@param {type:"boolean"}

  #@markdown **Prompt Settings**
  seed = 42000 #@param

  ##@markdown *One prompt by line*
  #prompts = "a forest by asher brown durand" #@param {type:"raw"}

  #@markdown **Image Settings**
  n_samples = 1 #@param
  n_rows = 2 #@param
  W = 512 #@param
  H = 512 #@param

  #@markdown **Init Settings**
  use_init = False #@param {type:"boolean"}
  init_image = "videoFrames/0001.jpg" #@param {type:"string"}
  strength = 0.5 #@param {type:"number"}
  
  #@markdown **Sampling Settings**
  scale = 10 #@param
  n_iter = 4 #@param
  ddim_steps = 50 #@param
  ddim_eta = 1.0 #@param
  #plms = False #@param {type:"boolean"}
  #k_lms = True #@param {type:"boolean"}

  sampler = "plms" # @param ["ddim","plms","k_lms","dpm_2","dpm_2_ancestral","heun","euler","euler_ancestral"] 
  
  #@markdown **Batch Settings**
  n_batch = 1 #@param

  #@markdown **no idea what this does**
  precision = 'autocast' #@param
  fixed_code = True #@param
  C = 4 #@param
  f = 8 #@param


  #@markdown **Animation Settings**

#@markdown ####**Animation Mode:**
  animation_mode = 'None' #@param ['None', '2D', 'Video Input'] {type:'string'}
  #@markdown IMPORTANT: For animation you must choose ddim or plms as sampler
  ### code must be changed to accept other sampler (issue with the "skip steps")
  
  #@markdown ---

  #@markdown ####**Video Input Settings:**

  video_init_path = "/mnt/c/Users/Thiba/Downloads/output_colab/test_video.mp4" #@param {type: 'string'}
  extract_nth_frame = 1 #@param {type: 'number'}
  video_init_seed_continuity = True #@param {type: 'boolean'}

  if animation_mode == "Video Input":
    if mount_google_drive:
      videoFramesFolder = f'/content/videoFrames'
    else:
      videoFramesFolder = f'videoFrames'
    os.makedirs(videoFramesFolder, exist_ok=True)
    print(f"Exporting Video Frames (1 every {extract_nth_frame})...")
    try:
      for f in pathlib.Path(f'{videoFramesFolder}').glob('*.jpg'):
        f.unlink()
    except:
      print('')
    vf = f'select=not(mod(n\,{extract_nth_frame}))'
    subprocess.run(['ffmpeg', '-i', f'{video_init_path}', '-vf', f'{vf}', '-vsync', 'vfr', '-q:v', '2', '-loglevel', 'error', '-stats', f'{videoFramesFolder}/%04d.jpg'], stdout=subprocess.PIPE).stdout.decode('utf-8')

  #@markdown ---

  #@markdown ####**2D Animation Settings:**
  #@markdown `zoom` is a multiplier of dimensions, 1 is no zoom.
  #@markdown All rotations are provided in degrees.

  key_frames = True #@param {type:"boolean"}
  max_frames = 48#@param {type:"number"}
  
  if animation_mode == "Video Input":
    max_frames = len(glob(f'{videoFramesFolder}/*.jpg'))

  interp_spline = 'Linear' #Do not change, currently will not look good. param ['Linear','Quadratic','Cubic']{type:"string"}
  angle = "0:(0)"#@param {type:"string"}
  zoom = "0: (1.0)"#@param {type:"string"}
  translation_x = "0: (0)"#@param {type:"string"}
  translation_y = "0: (0)"#@param {type:"string"}
  near_plane = 200#@ param {type:"number"}
  far_plane = 10000#@ param {type:"number"}
  fov = 40# @ param {type:"number"}
  padding_mode = 'border'#@ param {type:"string"}
  sampling_mode = 'bicubic'#@ param {type:"string"}



  #@markdown ---

  #@markdown ####**Coherency Settings:**
  #@markdown `strength_previous_frame` higher value change more the frame
  strength_previous_frame = 0.50 #@param {type:"number"}


  return locals()
  

In [ ]:
prompts = [
#"a beautiful forest by Asher Brown Durand, trending on Artstation", #the first prompt I want
"a beautiful portrait of a goddess, by Artgerm and Mucha and Greg Rutkowski, trending on Artstation", #the second prompt I want
#["the third prompt I don't want it I commented it with an #"],
]

In [ ]:
#@markdown **Run**
params = opt_params()
params["filename"] = None
params["prompts"] = prompts
for ii in range(params["n_batch"]):
  num = params["n_batch"]
  print(f"run {ii+1} of {num}")
  run(params)